# Curs IA  Octubre 2021 - Pràctica
## Part de definició del model per la paràctica de classificació de departament
**Proves amb les dades classificades i resumides per tenir menys categories**

Els requeriments per executar aquest quadern són els següents:
- Arxiu model.bin ubicat a la carpeta "./34/". Conté el model amb el vector de característiques de gairebé 800mil paraules en català. Aquest arxiu l'obtenim de la pàgina http://vectors.nlpl.eu/repository/, concretament de la URL http://vectors.nlpl.eu/repository/20/34.zip
- Arxiu tweets.csv. Aquest arxiu conté els tweets obtinguts.


L'arxiu tweets.csv haurà de contenir, com a mínim, les columnes:
- text
- classe


La columna classe haurà d'estar codificada de 0 a N - 1, essent N el nombre total de classes. En el nostre cas departaments

In [ ]:
# Aquest codi permet descarregar el model en català del vector de característiques de cada paraula

!wget http://vectors.nlpl.eu/repository/20/34.zip
!unzip 34.zip
!ls
!rm 34.zip

--2021-11-25 13:06:32--  http://vectors.nlpl.eu/repository/20/34.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.181
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 602902722 (575M) [application/zip]
Saving to: ‘34.zip’

34.zip              100%[===================>] 574.97M  25.7MB/s    in 23s     

2021-11-25 13:06:56 (25.0 MB/s) - ‘34.zip’ saved [602902722/602902722]

Archive:  34.zip
  inflating: LIST                    
  inflating: meta.json               
  inflating: model.bin               
  inflating: model.txt               
  inflating: README                  
34.zip		    LIST       model.bin  README
dadespracresum.tsv  meta.json  model.txt  sample_data


## Referències

- Exmple de classificació de textos fent servir CNN: https://cezannec.github.io/CNN_Text_Classification/
- Corpus d'idiomes amb els seus models d'embedding: http://vectors.nlpl.eu/repository/

## Inici

Comencem important les llibreries necessàries

In [ ]:
import pandas as pd                                   # tractament d'arxius i dades
import re                                             # mòdul d'expressions regulars
import numpy as np                                    # llibreria multifuncional (visualització i tractament de daddes)

from sklearn.model_selection import train_test_split  # funció que ens permetrà dividir les dades en dades d'entrenament, de test i de validació

from sklearn import preprocessing                     # la farem servir per passar els valors de la columna que conté les
                                                      # etiquetes (en el nostre cas els departaments) a valors numèrics del
                                                      # 0 al N-1 departaments.
                                                      # si l'arxiu que conté les dades d'entrenament ja conté la columna
                                                      # de les etiquetes codificades entre 0 i N-1 no caldrà fer-la servir

from gensim.models import KeyedVectors                # funció que permet carregar un model d'embedding d'un idioma
                                                      # aquest model conté el vector de característiques d'un diccionari
                                                      # de paraules prèviament entrenat.
                                                      # en el nostre cas fem servir un corpus en català amb quasi 800mil
                                                      # paraules

import torch                                          # llibreria d'aprenentatge automàtic
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F

## Tractament de les dades
### Obtenció de les dades

Llegim l'arxiu CSV amb els tweets i mostrem informació sobre les dades

In [ ]:
dades = pd.read_csv('dadespracresum.tsv', sep='\t')
dades.shape

(475, 7)

In [ ]:
dades.head()

,arxiu,hash,classet,classe,classer,text_orig,text
0,bcn_cat_tweet_20211021_203425.csv,8322723a13e516aeb6b9a358f757de02ad8c105ae0aa0f...,5 Urbanisme,5,2,RT @aitooru: @bcn_ajuntament Villaroel amb Gra...,rt villaroel amb gran viaabans aparcaments a...
1,bcn_cat_tweet_20211021_203425.csv,dd4677846d72141cced6f34ae1f02a3403968c01eba79f...,5 Urbanisme,5,2,RT @Dr_diez: @bcn_ajuntament Deixar ela carrer...,rt deixar ela carrers pagats de formigó comp...
2,bcn_cat_tweet_20211021_203425.csv,86c491271d5d456f27fe375b0cc0ce9323eb8085cd5d6c...,2 Politica,2,1,@bcn_ajuntament Heu destrossat Barcelona! Cost...,heu destrossat barcelona costarà anys reconstr...
3,bcn_cat_tweet_20211021_203425.csv,61a17072a12a1d64813327df9a2b1a0e6f609b8e8b446a...,3 Serveis Municipals,3,2,RT @AntoniaGiro: @bcn_ajuntament Si treballes ...,rt si treballes en precari i mal pagat en un...
4,bcn_cat_tweet_20211021_203425.csv,a6943dc1c60ba6699cb6c2b9eed8931ddd10737a6e6c24...,8 OAC,8,3,"@bcn_ajuntament , bona tarde, si us plau em pr...",bona tarde si us plau em pregunta ma germana q...


Ens quedem amb les columnes que ens interessen

In [ ]:
dades_utils = dades.drop(['arxiu', 'hash', 'text_orig','classet','classe'], axis=1)
dades_utils.head()

,classer,text
0,2,rt villaroel amb gran viaabans aparcaments a...
1,2,rt deixar ela carrers pagats de formigó comp...
2,1,heu destrossat barcelona costarà anys reconstr...
3,2,rt si treballes en precari i mal pagat en un...
4,3,bona tarde si us plau em pregunta ma germana q...


Definim una funció per netejar les dades. Traurem les emoticones, hashtags, referències a altres comptes de Tweeter, URLs, etc.

In [ ]:
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
def neteja_text(x):
    txt = x
    # treu emojis
    txt = emoji_pattern.sub(r' ', txt)
    # treu les referències a altres comptes (@usuari)
    txt = re.sub("@\S+", "", txt)
    # treu els hashtags
    txt = re.sub("#\S+", "", txt)
    # treu el text "RT"
    txt = re.sub(r'^rt[\s]+', '', txt)
    # treu els hiperenllaços
    txt = re.sub(r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()!@:%_\+.~#?&\/\/=]*)', '', txt)
    txt = re.sub(r"[!#$%&'()*+,\-./:;<=>?@[\\\]^_`{|}~]", ' ', txt)
    # minúscules
    txt = txt.lower()
    return txt

In [ ]:
dades_utils['text_net'] = dades_utils['text'].apply(neteja_text)
dades_utils.head()

,classer,text,text_net
0,2,rt villaroel amb gran viaabans aparcaments a...,villaroel amb gran viaabans aparcaments a la c...
1,2,rt deixar ela carrers pagats de formigó comp...,deixar ela carrers pagats de formigó comprat a...
2,1,heu destrossat barcelona costarà anys reconstr...,heu destrossat barcelona costarà anys reconstr...
3,2,rt si treballes en precari i mal pagat en un...,si treballes en precari i mal pagat en una emp...
4,3,bona tarde si us plau em pregunta ma germana q...,bona tarde si us plau em pregunta ma germana q...


### Tokenització de les dades
Carreguem el model amb el vector de característiques en català i convertim a tokens els textos llegits de l'arxiu d'entrada.
Si no trobem la paraula al corpus li assignem un 0.

In [ ]:
model_embedding = KeyedVectors.load_word2vec_format('model.bin', binary = True)

def tokenitza_textos(model_embedding, textos):
    paraules = [tweet.split() for tweet in textos]

    textos_tokenitzats = []
    for tweet in paraules:
        valors = []
        for paraula in tweet:
            try:
                index = model_embedding.vocab[paraula].index
            except: 
                index = 0
            valors.append(index)
        textos_tokenitzats.append(valors)
    
    return textos_tokenitzats

tt = tokenitza_textos(model_embedding, dades_utils['text_net'])

In [ ]:
 dades_utils['text_net']

0      villaroel amb gran viaabans aparcaments a la c...
1      deixar ela carrers pagats de formigó comprat a...
2      heu destrossat barcelona costarà anys reconstr...
3      si treballes en precari i mal pagat en una emp...
4      bona tarde si us plau em pregunta ma germana q...
                             ...                        
470    tanmateix et demanem que esborris i no pengis ...
471    per aquest motiu és que en cas dapagada aconse...
472    actualment continuen les revisions dels equips...
473    daquesta zona de lenllumenat públic i no és se...
474                    lo de baixar el ibi per altre dia
Name: text_net, Length: 475, dtype: object

### Normalització de la matriu d'entrada del model
Tenint en compte que la CNN necessita una quantitat fixa de columnes d'entrada hem d'assegurar que la matriu que representa els textos i els tokens tingui un nombre de columnes determinat. Aquest procés es coneix com a padding.

La matriu resultant del padding conté la X del model.

In [ ]:
def padding(tt, longitud):
    caract = np.zeros((len(tt), longitud), dtype=int)

    for i, fila in enumerate(tt):
        caract[i, -len(fila):] = np.array(fila)[:longitud]
    
    return caract

# Definim una matriu amb 200 columnes
X = padding(tt, 200)

### Preparar etiquetes
Convertim el tipus de dades que conté les etiquetes

In [ ]:
etiquetes = dades_utils['classer'].to_numpy()

## Model CNN

### Preparació de les dades d'entrada del model

#### Divisió de les dades d'entrada en entrenament, test i validació

In [ ]:
train_x, X_resta, train_y, y_resta = train_test_split(X,
                                                    etiquetes,
                                                    test_size = 0.3,
                                                    random_state = 0)

# dividim la resta de dades que no són entrenament en test i validació
test_idx = int(len(X_resta) * 0.5)
val_x, test_x = X_resta[:test_idx], X_resta[test_idx:]
val_y, test_y = y_resta[:test_idx], y_resta[test_idx:]

#### Conversió de les dades d'entrada del model

In [ ]:
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

batch_size = 50

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

### Definició de l'arquitectura del model

Abans comprovem si està disponible el motor d'execució de la GPU

In [ ]:
train_on_gpu = torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

No GPU available, training on CPU.


In [ ]:

class ClassificaCNN(nn.Module):
    """
     model_embedding:              model que conté el corpus de l'idioma (aquest conté el vector de N caracterísques de cada paraula)
     mida_vocabulari:              quantitat de paraules del corpus
     mida_sortida:                 quantitat de resultats, en aquest cas és el número de classes
     mida_vector_caracteristiques: longitud del vector de característiques del corpus
     num_filtres:                  nombre de filtres que es faran servir a la convolució
     mides_kernels:                mides dels kernels a aplicar ==> els kernels seran de:
                                      [3, 100], [4, 100] i [5, 100]    ==> [3 o 4 o 5, mida_vector_caracteristiques]
     freeze_embeddings:            documentació oficial "If True, the tensor does not get updated in the learning process"
     drop_prob:                    probabilitat a aplicar a la capa de dropout
    """
    def __init__(self, 
                 model_embedding,
                 mida_vocabulari,
                 mida_sortida,
                 mida_vector_caracteristiques,
                 num_filtres = 100,
                 mides_kernels = [3, 4, 5],
                 freeze_embeddings = True,
                 drop_prob = 0.5):

        super(ClassificaCNN, self).__init__()

        self.num_filtres = num_filtres
        self.mida_vector_caracteristiques = mida_vector_caracteristiques
        
        # 1. capa d'embedding
        self.embedding = nn.Embedding(mida_vocabulari, mida_vector_caracteristiques)
        #    li passem els pesos del model_embedding a la capa
        self.embedding.weight = nn.Parameter(torch.from_numpy(model_embedding.vectors))
        #    (opcional) Documentació oficial "If True, the tensor does not get updated in the learning process"
        if freeze_embeddings:
            self.embedding.requires_grad = False
        
        # 2. capes convolucionals
        #    Es creen tantes capes convolucionals com kernels vulguem, per defecte són 3.
        #    L'entrada de cada capa és 1: 1 paraula
        #    La sortida de cada cap és igual a la mida del vector de característiques del corpus, normalment 100, 200 o 300
        self.convs_1d = nn.ModuleList([nn.Conv2d(1, 
                                                 num_filtres, 
                                                 (k, mida_vector_caracteristiques),    # [3, 100], [4, 100] i [5, 100]
                                                 padding = (k - 2, 0))                 # (1, 0), (2, 0) i (3, 0)
                                       for k in mides_kernels
                                      ]
                                     )
        
        # 3. capa fully-connected per la classificació final
        self.fc = nn.Linear(len(mides_kernels) * num_filtres, mida_sortida) 
        
        # 4. capa de dropout
        self.dropout = nn.Dropout(drop_prob)
        
    
    def conv_and_pool(self, x, conv):
        x = F.relu(conv(x)).squeeze(3)
        
        x_max = F.max_pool1d(x, x.size(2)).squeeze(2)
        return x_max

    def forward(self, x):
        embeds = self.embedding(x)
        embeds = embeds.unsqueeze(1)
        
        conv_results = [self.conv_and_pool(embeds, conv) for conv in self.convs_1d]
        
        x = torch.cat(conv_results, 1)
        x = self.dropout(x)
        
        # fem un flatten de la matriu a vector
        x = x.view(-1, len(mides_kernels) * num_filtres)
        
        x = self.fc(x)
        return x

Definim els hiperparàmetres

In [ ]:
mida_vocabulari = len(model_embedding.index2word)           # quantitat de paraules del corpus en català

mida_sortida = 4                                          # mida del resultat. 
                                                            # ÉS MOLT IMPORTANT QUE LES DADES DE LES ETIQUETES y_train, y_test 
                                                            # tinguin els valors compressos entre 0 i (mida_sortida - 1) de cara
                                                            # a calcular la funció de costos CrossEntropyLoss
        
mida_vector_caracteristiques = model_embedding.vector_size  # longitud del vector de característiques del corpus en català
num_filtres = 100                                           # nombre de filtres que es faran servir a la convolució
mides_kernels = [3, 4, 5]                                   # mides dels kernels a aplicar

net = ClassificaCNN(model_embedding, mida_vocabulari, mida_sortida, mida_vector_caracteristiques, num_filtres, mides_kernels)
                    
print(net)

ClassificaCNN(
  (embedding): Embedding(799020, 100)
  (convs_1d): ModuleList(
    (0): Conv2d(1, 100, kernel_size=(3, 100), stride=(1, 1), padding=(1, 0))
    (1): Conv2d(1, 100, kernel_size=(4, 100), stride=(1, 1), padding=(2, 0))
    (2): Conv2d(1, 100, kernel_size=(5, 100), stride=(1, 1), padding=(3, 0))
  )
  (fc): Linear(in_features=300, out_features=4, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


## Entrenament del model

Definim la ràtio d'entrenament, la funció de costos i l'optimitzador

In [ ]:
lr = 0.001

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr = lr)

In [ ]:
def train(net, train_loader, epochs):

    if(train_on_gpu):
        net.cuda()

    valid_loss_min = np.Inf

    for epoch in range(1, epochs+1):
        train_loss = 0.0
        valid_loss = 0.0

        net.train()
        for batch_idx, (inputs, labels) in enumerate(train_loader):
            if train_on_gpu:
                inputs, labels = inputs.cuda(), labels.cuda()

            net.zero_grad()
            
            inputs = inputs.type(torch.LongTensor)
            output = net(inputs)
            loss = criterion(output.squeeze(), labels.long())
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * inputs.size(0)

        net.eval()
        for batch_idx, (inputs, labels) in enumerate(valid_loader):
            if train_on_gpu:
                inputs, labels = inputs.cuda(), labels.cuda()

            inputs = inputs.type(torch.LongTensor)
            output = net(inputs)
            loss = criterion(output.squeeze(), labels.long())
            valid_loss += loss.item() * inputs.size(0)

        train_loss = train_loss/len(train_loader.sampler)
        valid_loss = valid_loss/len(valid_loader.sampler)

        print('Època: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch, train_loss, valid_loss))

        if valid_loss <= valid_loss_min:
            print('Validation loss disminueix ({:.6f} --> {:.6f}).  Desant el model ...'.format(valid_loss_min, valid_loss))
            torch.save(net.state_dict(), 'model_classificacio.pt')
            valid_loss_min = valid_loss

In [ ]:
epochs = 20

train(net, train_loader, epochs)

Època: 1 	Training Loss: 1.216590 	Validation Loss: 1.376654
Validation loss disminueix (inf --> 1.376654).  Desant el model ...
Època: 2 	Training Loss: 1.042200 	Validation Loss: 1.344534
Validation loss disminueix (1.376654 --> 1.344534).  Desant el model ...
Època: 3 	Training Loss: 0.975907 	Validation Loss: 1.246694
Validation loss disminueix (1.344534 --> 1.246694).  Desant el model ...
Època: 4 	Training Loss: 0.901334 	Validation Loss: 1.233034
Validation loss disminueix (1.246694 --> 1.233034).  Desant el model ...
Època: 5 	Training Loss: 0.840562 	Validation Loss: 1.187497
Validation loss disminueix (1.233034 --> 1.187497).  Desant el model ...
Època: 6 	Training Loss: 0.760701 	Validation Loss: 1.180666
Validation loss disminueix (1.187497 --> 1.180666).  Desant el model ...
Època: 7 	Training Loss: 0.726488 	Validation Loss: 1.162419
Validation loss disminueix (1.180666 --> 1.162419).  Desant el model ...
Època: 8 	Training Loss: 0.632538 	Validation Loss: 1.133506
Valida

## Test del model

Carreguem el model desat a la fase d'entrenament

In [ ]:
net.load_state_dict(torch.load('model_classificacio.pt'))

<All keys matched successfully>

Definim el vector que conté les nostres classes

In [ ]:
classes = ["Altres","Politica i hisenda","Ciutat","Ciutada i Seguretat"]

In [ ]:
test_loss = 0.0
class_correct = list(0. for i in range(mida_sortida))
class_total = list(0. for i in range(mida_sortida))

net.eval()

for batch_idx, (inputs, labels) in enumerate(test_loader):
    if train_on_gpu:
        inputs, labels = inputs.cuda(), labels.cuda()

    inputs = inputs.type(torch.LongTensor)
    output = net(inputs)
    loss = criterion(output.squeeze(), labels.long())
    test_loss += loss.item() * inputs.size(0)
    _, pred = torch.max(output, 1)    
    correct_tensor = pred.eq(labels.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    for i in range(len(inputs)):
        label = labels.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# average test loss
test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(mida_sortida):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

Test Loss: 0.900101

Test Accuracy of Altres: 33% ( 1/ 3)
Test Accuracy of Politica i hisenda: 41% ( 7/17)
Test Accuracy of Ciutat: 93% (40/43)
Test Accuracy of Ciutada i Seguretat:  0% ( 0/ 9)

Test Accuracy (Overall): 66% (48/72)


## Part de desplegament del model amb Gradio.app


Instal·lem el Gradio i carreguem la llibreria

In [ ]:
!pip install gradio
import gradio as gr

     |████████████████████████████████| 979 kB 27.4 MB/s 
     |████████████████████████████████| 1.9 MB 37.8 MB/s 
     |████████████████████████████████| 206 kB 44.9 MB/s 
     |████████████████████████████████| 961 kB 31.8 MB/s 
     |████████████████████████████████| 3.6 MB 34.7 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
  Created wheel for ffmpy: filename=ffmpy-0.3.0-py3-none-any.whl size=4710 sha256=a39dfc608b93875bace8e955bc0d955f8d26e7e3af52716c79151be84cf9defc
  Stored in directory: /root/.cache/pip/wheels/13/e4/6c/e8059816e86796a597c6e6b0d4c880630f51a1fcfa0befd5e6
  Created wheel for flask-cachebuster: filename=Flask_CacheBuster-1.0.0-py3-none-any.whl size=3371 sha256=9eee643eb7efe119088303ef42e625b6671d4838280cb5a99a4c66f8fdb59198
  Stored in directory: /root/.cache/pip/wheels/28/c0/c4/44687421dab41455be93112bd1b0dee1f3c5a9aa27bee63708
Successfully built ffmpy flask-cachebuster


Despleguem l'app a gradio

Repetim alguns dels passos anteirors aquí com a recordatori de que s'han de fer en un desplegament separat de l'entrenament

In [ ]:
classes = ["Altres","Politica i hisenda","Ciutat","Ciutada i Seguretat"]

In [ ]:
net.load_state_dict(torch.load('model_classificacio.pt'))

emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
def neteja_text(x):
    txt = x
    # treu emojis
    txt = emoji_pattern.sub(r' ', txt)
    # treu les referències a altres comptes (@usuari)
    txt = re.sub("@\S+", "", txt)
    # treu els hashtags
    txt = re.sub("#\S+", "", txt)
    # treu el text "RT"
    txt = re.sub(r'^rt[\s]+', '', txt)
    # treu els hiperenllaços
    txt = re.sub(r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()!@:%_\+.~#?&\/\/=]*)', '', txt)
    txt = re.sub(r"[!#$%&'()*+,\-./:;<=>?@[\\\]^_`{|}~]", ' ', txt)
    # minúscules
    txt = txt.lower()
    return txt

model_embedding = KeyedVectors.load_word2vec_format('model.bin', binary = True)

def tokenitza_textos(model_embedding, textos):
    paraules = [tweet.split() for tweet in textos]

    textos_tokenitzats = []
    for tweet in paraules:
        valors = []
        for paraula in tweet:
            try:
                index = model_embedding.vocab[paraula].index
            except: 
                index = 0
            valors.append(index)
        textos_tokenitzats.append(valors)
    
    return textos_tokenitzats

def padding(tt, longitud):
    caract = np.zeros((len(tt), longitud), dtype=int)

    for i, fila in enumerate(tt):
        caract[i, -len(fila):] = np.array(fila)[:longitud]
    
    return caract


Definim la funció que farà la classificació i que passarem al gradio

In [ ]:
def classifica_tweets(el_tweet):
    tweet = []
    tweet = [neteja_text(el_tweet)]
    #print('el tweet: ', tweet)
    tt = tokenitza_textos(model_embedding, tweet)
    #print('tt: ', tt)
    X = padding(tt, 200)
    #print('X: ', X)

    net.eval()
    X = torch.from_numpy(X).type(torch.LongTensor)
    if train_on_gpu:
        net().cuda()
        X = X.cuda()

    output = F.softmax(net(X), dim=1)
    #print('output: ', output)

    return {classes[i]: float(output[0][i]) for i in range(len(classes))}

Despleguem el model a gradio:

In [ ]:
inputs = gr.inputs.Textbox(lines=5, label="Enganxa el Tweet aquí: ")
outputs = gr.outputs.Label(num_top_classes=3)
gr.Interface(fn=classifica_tweets, inputs=inputs, outputs=outputs).launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://51023.gradio.app

This share link will expire in 72 hours. To get longer links, send an email to: support@gradio.app


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7860/',
 'https://51023.gradio.app')

**Fins aquí l'execució de la pàctica**

# Notebook utilitzat per l'obtenció dels tweets

In [ ]:
# coding:utf-8

import tweepy
import csv
import time
from datetime import datetime, date, timedelta
import re

#Obtenim data actual
today = datetime.today()
#Especifiquem un rang de dates per obtenir tweets (Exemple 20 dies)
tweet_begin_date = datetime.strftime(today - timedelta(days=20), '%Y-%m-%d_00:00:00')
tweet_end_date = datetime.strftime((today), '%Y-%m-%d_23:59:00')


# Guardem CSV a arrel del projecte
csv_dir = './'

# Twitter API KEY - Obtingudes servei API Twitter, user @regonzalezmas
Consumer_key = "MGVRn15j2DLuDXmimfKV0s2vs"
Consumer_secret = "Ah8a6HkFvgsWkwAS3MY7uftq8Rh6TIPUQKOwxw8H9rjVCikxtk"
Access_token ="240674330-l6x04wZ59zJni7Aoy37H5re9raCDYPkJrZH3wuFS"
Access_secret = "zyJsBlM4SgpflawPFOYd9bUZNq0zyxaArLN2dWjIPhGvR"


#Autenticació Twitter API 
def authTwitter():
        auth = tweepy.OAuthHandler(Consumer_key, Consumer_secret)
        auth.set_access_token(Access_token, Access_secret)
        api = tweepy.API(auth, retry_count=3,retry_delay=40,retry_errors=set([401, 404, 500, 502, 503, 504]), wait_on_rate_limit = True, wait_on_rate_limit_notify=True)
        return(api)

#Funció per obtenir tweets
def get_tweet(s):
        api = authTwitter() #Auth
        tweet_list = []
        tweet_id_list = []
        user_id_list = []

        tweets = tweepy.Cursor(api.search, q = s,     #String cerca
                 include_entities = True,   
                 tweet_mode = 'extended',   
                 since = tweet_begin_date,    
                 lang = 'ca').items()       #Idioma català

        #Guardem tweet a una llista
        for tweet in tweets:
                tweet_list.append([tweet.id, tweet.user.screen_name, tweet.created_at, tweet.full_text.replace('\n',''), tweet.favorite_count, tweet.retweet_count])
                tweet_id_list.append(tweet.id)
                user_id_list.append(tweet.user.screen_name)
        #Sortida en fitxer csv
        with open(csv_dir+'tweet_'+ today.strftime('%Y%m%d_%H%M%S') + '.csv', 'w',newline='',encoding='utf-8') as f:
                writer = csv.writer(f, lineterminator='\n')
                writer.writerow(["id","user","created_at","text","fav","RT"])
                writer.writerows(tweet_list)
        pass

def main():
        #get_tweet("(to:vallsajuntament OR to:reus_cat OR to:elvendrell_cat) lang:ca")
        get_tweet("(to:bcn_ajuntament) lang:ca")
        #get_tweet("(to:barcelona_010) lang:ca")
        #get_tweet("(to:TGNAjuntament) lang:ca")
        #get_tweet("(to:paerialleida) lang:ca")
        #get_tweet("(to:girona_cat) lang:ca")
        #get_tweet("(to:012) lang:ca")
        #get_tweet("(to:gencat) lang:ca")

if __name__ == "__main__":
        main()

# Notebook utilitzat per el filtratge i generació de l'arxiu de dades de prova

# Curs IA - Paràctica filtratge de dades
Part de neteja de dades per la paràctica de classificació de departament
### Generació dels CSV 
Generarà un arxiu csv amb el filtrat del tots els tweets sota diferents criteris, columnes resultat:
* arxiu: arxiu origen
* hash: Per identificar el tweet en cas de tenir feina a mitges i no tornar a classificar lo fet
* classe: aquí caldrà posar manualment la classe (del desplegable donat)
* text_orig:Text original del tweet (origen del hash)
* text: text complet ja netejat


## Importacio inicial
Cal importar a la carpeta **d1** les dades de tots els csv que volguem importar

In [ ]:
!ls d1


bcn_cat_tweet_20211021_203425.csv  paerialleida_20211021_204148.csv
girona_tweet_20211021_204406.csv   TGNAjuntament_tweet_20211021_203819.csv


Ara creem la classe que ens gestionarà la importació, neteja i tokenització dels CSV i ens generarà un sol arxiu csv amb les dades com les volem

Particularitat:
* Millorat el filtre de emojis
* Afegit usuarisprohibits amb els usuaris de que no volem agafar els tweets ja que son origen  i no respostes
* Trets els noms que comencen amb @ per ser referencies no útils
* Evito duplicats


In [ ]:
import csv
import nltk  
from nltk import tokenize
import string
import re
import hashlib
from os import listdir
from os.path import join,isfile

#http://latel.upf.edu/morgana/altres/pub/ca_stop.htm
sepcatala="',a,abans,abans-d'ahir,abintestat,ací,adesiara,adés,adéu,adàgio,ah,ahir,ai,aitambé,aitampoc,aitan,aitant,aitantost,aixà,això,així,aleshores,algun,alguna,algunes,alguns,algú,alhora,allà,allèn,allò,allí,almenys,alto,altra,altre,altres,altresí,altri,alça,al·legro,amargament,amb,ambdues,ambdós,amunt,amén,anc,andante,andantino,anit,ans,antany,apa,aprés,aqueix,aqueixa,aqueixes,aqueixos,aqueixs,aquell,aquella,aquelles,aquells,aquest,aquesta,aquestes,aquests,aquèn,aquí,ara,arran,arrera,arrere,arreu,arri,arruix,atxim,au,avall,avant,aviat,avui,açò,bah,baix,baldament,ballmanetes,banzim-banzam,bastant,bastants,ben,bis,bitllo-bitllo,bo,bé,ca,cada,cal,cap,car,caram,catorze,cent,centes,cents,cerca,cert,certa,certes,certs,cinc,cinquanta,cinquena,cinquenes,cinquens,cinquè,com,comsevulla,contra,cordons,corrents,cric-crac,d,daixonses,daixò,dallonses,dallò,dalt,daltabaix,damunt,darrera,darrere,davall,davant,de,debades,dedins,defora,dejorn,dejús,dellà,dementre,dempeus,demés,demà,des,desena,desenes,desens,després,dessobre,dessota,dessús,desè,deu,devers,devora,deçà,diferents,dinou,dins,dintre,disset,divers,diversa,diverses,diversos,divuit,doncs,dos,dotze,dues,durant,ecs,eh,el,ela,elis,ell,ella,elles,ells,els,em,emperò,en,enans,enant,encara,encontinent,endalt,endarrera,endarrere,endavant,endebades,endemig,endemés,endemà,endins,endintre,enfora,engir,enguany,enguanyasses,enjús,enlaire,enlloc,enllà,enrera,enrere,ens,ensems,ensota,ensús,entorn,entre,entremig,entretant,entrò,envers,envides,environs,enviró,ençà,ep,ep,era,eren,eres,ergo,es,escar,essent,esser,est,esta,estada,estades,estan,estant,estar,estaran,estarem,estareu,estaria,estarien,estaries,estaré,estarà,estaràs,estaríem,estaríeu,estat,estats,estava,estaven,estaves,estem,estes,esteu,estic,estiguem,estigueren,estigueres,estigues,estiguessis,estigueu,estigui,estiguin,estiguis,estigué,estiguérem,estiguéreu,estigués,estiguí,estos,està,estàs,estàvem,estàveu,et,etc,etcètera,ets,excepte,fins,fora,foren,fores,força,fos,fossin,fossis,fou,fra,fui,fóra,fórem,fóreu,fóreu,fóssim,fóssiu,gaire,gairebé,gaires,gens,girientorn,gratis,ha,hagi,hagin,hagis,haguda,hagudes,hagueren,hagueres,haguessin,haguessis,hagut,haguts,hagué,haguérem,haguéreu,hagués,haguéssim,haguéssiu,haguí,hala,han,has,hauran,haurem,haureu,hauria,haurien,hauries,hauré,haurà,hauràs,hauríem,hauríeu,havem,havent,haver,haveu,havia,havien,havies,havíem,havíeu,he,hem,heu,hi,ho,hom,hui,hàgim,hàgiu,i,igual,iguals,inclusive,ja,jamai,jo,l,la,leri-leri,les,li,lla,llavors,llevat,lluny,llur,llurs,lo,los,ls,m,ma,mai,mal,malament,malgrat,manco,mant,manta,mantes,mantinent,mants,massa,mateix,mateixa,mateixes,mateixos,me,mentre,mentrestant,menys,mes,meu,meua,meues,meus,meva,meves,mi,mig,mil,mitges,mitja,mitjançant,mitjos,moixoni,molt,molta,moltes,molts,mon,mos,més,n,na,ne,ni,ningú,no,nogensmenys,només,noranta,nos,nosaltres,nostra,nostre,nostres,nou,novena,novenes,novens,novè,ns,nòs,nós,o,oh,oi,oidà,on,onsevulga,onsevulla,onze,pas,pengim-penjam,per,perquè,pertot,però,piano,pla,poc,poca,pocs,poques,potser,prest,primer,primera,primeres,primers,pro,prompte,prop,prou,puix,pus,pàssim,qual,quals,qualsevol,qualsevulla,qualssevol,qualssevulla,quan,quant,quanta,quantes,quants,quaranta,quart,quarta,quartes,quarts,quasi,quatre,que,quelcom,qui,quin,quina,quines,quins,quinze,quisvulla,què,ran,re,rebé,renoi,rera,rere,res,retruc,s,sa,salvament,salvant,salvat,se,segon,segona,segones,segons,seguida,seixanta,sempre,sengles,sens,sense,ser,seran,serem,sereu,seria,serien,series,seré,serà,seràs,seríem,seríeu,ses,set,setanta,setena,setenes,setens,setze,setè,seu,seua,seues,seus,seva,seves,si,sia,siau,sic,siguem,sigues,sigueu,sigui,siguin,siguis,sinó,sis,sisena,sisenes,sisens,sisè,sobre,sobretot,sol,sola,solament,soles,sols,som,son,sos,sota,sots,sou,sovint,suara,sí,sóc,són,t,ta,tal,tals,també,tampoc,tan,tanmateix,tant,tanta,tantes,tantost,tants,te,tercer,tercera,terceres,tercers,tes,teu,teua,teues,teus,teva,teves,ton,tos,tost,tostemps,tot,tota,total,totes,tothom,tothora,tots,trenta,tres,tret,tretze,tu,tururut,u,uf,ui,uix,ultra,un,una,unes,uns,up,upa,us,va,vagi,vagin,vagis,vaig,vair,vam,van,vares,vas,vau,vem,verbigràcia,vers,vet,veu,vint,vora,vos,vosaltres,vostra,vostre,vostres,vostè,vostès,vuit,vuitanta,vuitena,vuitenes,vuitens,vuitè,vés,vàreig,vàrem,vàreu,vós,xano-xano,xau-xau,xec,érem,éreu,és,ésser,àdhuc,àlies,ça,ço,òlim,ídem,últim,última,últimes,últims,únic,única,únics,úniques".split(",")
#usuaris  que volem treure del twiter
usuarisprohibits="bcn_ajuntament,girona_cat,paerialleida,TGNAjuntament".lower().split(",")

emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", flags=re.UNICODE)
referencies=re.compile("@\S+")                           

nltk.download('punkt')

class GenCSV:
    stpwords=[]
    usuarispr=[]
    fets=[]
    def __init__(self,stopwords=sepcatala,usuarispr=usuarisprohibits) -> None:
        self.stpwords=stopwords 
        self.usproh=usuarispr        
    def IngestaCSV(self, carpeta,nf,dialecte={
            "delimiter":',',
            "doublequote":True,
            "quotechar":'"'
        }):
        rs=[]
        with open(join(carpeta,nf),encoding="utf-8") as f:
            for r in csv.DictReader(f,dialect=dialecte):
                #no els tweets propis
                if r["user"] in self.usproh:
                    continue
                tx=r["text"]                
                hsh=hashlib.sha256(tx.encode("utf-8")).hexdigest()
                #no frases repetides
                if hsh in self.fets:
                    continue
                self.fets.append(hsh)
                rs.append([nf,hsh,0,tx,self.netejaText(tx)])
        return rs
    def netejaText(self,text):
        #minucules
        txt=text.lower()
        #treiem emojis
        txt=emoji_pattern.sub(r' ', txt)
        #treiem arrobas
        txt=referencies.sub('',txt)        
        #puntuacio
        txt= txt.translate(str.maketrans('', '', string.punctuation)) # el maketrans ens fa una taula de traduccio de 1er joc de caracters al 2n (aqui en blanc) i el 3er parametre topts els caracters que eliminara        
        #els stopwords no els utilitzem ja que els tweets tenen molt poques paraules
        return txt
    def ProcessaCarpeta(self, carpeta,sortida):
        larx=[f for f in listdir(carpeta) if isfile(join(carpeta, f))]
        lns=[]
        for nf in larx:
            part=self.IngestaCSV(carpeta,nf)
            lns.extend(part)
        with open(sortida,'w',encoding="utf-8",newline='') as fsortida:
            writ=csv.writer(fsortida,delimiter='\t',lineterminator='\r\n')
            writ.writerow(['arxiu','hash','classe','text_orig','text'])
            for ar in lns:
                writ.writerow(ar)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Ara generem la sortida


In [ ]:
g=GenCSV()
print(g.ProcessaCarpeta("./d1","./sort.csv"))

None
